In [22]:
# !python -m pip install geopandas
import pandas as pd
import numpy as np
import geopandas as gpd

In [23]:
#reading rheast and extracting districts
rheas= pd.read_csv('data/rheas/zmb_n_25_chirps__newmask_1999_2023_maize.csv')
preharvest=pd.read_csv('data/preharvest/preharvest_cleaned.csv')
postharvest=pd.read_csv('data/postharvest/postharvest_cleaned.csv')


In [24]:
# # remove all white spaces, turn to lower case and
rheas.cname=rheas.cname.str.lower()
rheas.cname = rheas.cname.str.strip()
rheas.cname=rheas.cname.str.replace('-', '', regex=True)
rheas.cname=rheas.cname.str.replace(' ', '', regex=True)

In [25]:
#changing date columns
rheas['planting']=pd.to_datetime(rheas['planting'], format='ISO8601')
rheas['harvest'] = pd.to_datetime(rheas['harvest'], format='ISO8601')
rheas['year'] = rheas['planting'].dt.year
rheas['district']=rheas['cname']
rheas['yield']=rheas['gwad']/1000


In [26]:
#Derive seasons and add to rheas data
seasons = [1, 1, 1, 1,1, 0, 0, 0, 0, 1, 1, 1]
month_to_season=dict(zip(range(1,13), seasons))
rheas['season1']= rheas['planting'].dt.month.map(month_to_season)

# Extract data only for the planting season
tamsat_season= rheas.loc[rheas['season1']==1]

In [27]:
# aggregate data
tamsat_yearly=tamsat_season.groupby(['district', 'season1', 'year'], as_index=False).agg({'yield':'mean'})
preharvest_yearly=preharvest.groupby(['district', 'year'], as_index=False).agg({'yield':'mean'})

In [28]:
#Extracting Districts list
#first set cname to district
rheas['district']=rheas['cname']

preharvest_list_districts= preharvest['district'].to_list()
preharvest_set_districts=set(preharvest_list_districts)
preharvest_districts=list(preharvest_set_districts)
rheas_tamsat_list_districts=rheas['district'].to_list()
rheas_tamsat_set_districts=set(rheas_tamsat_list_districts)
rheas_districts=list(rheas_tamsat_set_districts)

In [29]:
#Finding common districts in rheas and preharvestharvest rheas
districts=[]
for i in preharvest_districts:
    if i in rheas_districts:
        districts.append(i)
    else:
        pass

missing_districts=[]
for j in rheas_districts:
    if j not in  preharvest_districts:
        missing_districts.append(j)
    else:
        pass
len(missing_districts)

42

In [30]:
# rheas and preharvest metrics
# preharvest is 2011 to 2022
rheas_preharvest_districts=districts
district_rheas_prehavest=tamsat_yearly.loc[(tamsat_yearly['year']>=2011)&(tamsat_yearly['year']<=2022)]
rheas_preharvest_disct_check=[]
distric_metrics1=[]

for i in rheas_preharvest_districts:
        district_preharvest=preharvest_yearly.loc[preharvest_yearly['district']==i].reset_index(drop=True)
        rheas_tamsat_data3=district_rheas_prehavest.loc[district_rheas_prehavest['district']==i].reset_index(drop=True)
        #  pearson's correlation
        corr=rheas_tamsat_data3['yield'].corr(district_preharvest['yield'])
        # # #root mean square error
        rmse = np.sqrt(((rheas_tamsat_data3['yield']-district_preharvest['yield'])**2).mean())
        # # #unbiased RMSE
        ubrmse = np.sqrt((((rheas_tamsat_data3['yield']-rheas_tamsat_data3['yield'].mean())
                    -(district_preharvest['yield']-district_preharvest['yield'].mean()))**2).mean())
        # #bias
        bias = (rheas_tamsat_data3['yield']-district_preharvest['yield']).mean()
        distric_metrics1.extend([[i, corr,rmse,ubrmse,bias]])
        # convert into dataframe and rename column
        rheas_preharvest_disct_check.append(district_preharvest)

check_rheas_preharvest_disct= pd.concat(rheas_preharvest_disct_check, ignore_index=True)
district_rheas_preharvest_metrics1=pd.DataFrame(distric_metrics1)
district_rheas_preharvest_metrics=district_rheas_preharvest_metrics1.rename({0:'district',1: 'correlation', 2: 
                                                    'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
district_rheas_preharvest_metrics

,district,correlation,rmse,ubrmse,bias
0,kalulushi,0.204006,0.902503,0.337987,0.836825
1,luangwa,-0.067729,2.909488,0.335867,2.890037
2,senanga,0.208361,3.053119,0.234977,3.044064
3,mazabuka,0.398194,1.336786,0.600286,1.194426
4,mkushi,0.247837,0.583331,0.570925,0.119665
...,...,...,...,...,...
69,mpika,0.277966,0.420741,0.249267,0.338953
70,mungwi,0.114521,0.646657,0.247954,0.597230
71,kafue,0.513453,1.184868,0.484386,1.081333
72,itezhitezhi,-0.479689,1.893106,0.664862,1.772515


In [31]:
# saven district metrics
district_rheas_preharvest_metrics.to_csv('data/metrics/preharvest_rheas_district_metrics.csv',index=False)

In [32]:
# Extract rheas data only for  districts in preharvest
districts
datasets=district_rheas_prehavest
district_rheas_pre_list=[]
for i in districts:
    rheast_pre_dist_df=district_rheas_prehavest.loc[district_rheas_prehavest['district']==i]
    district_rheas_pre_list.append(rheast_pre_dist_df)
rheas_pre_common=pd.concat(district_rheas_pre_list, ignore_index=True)
rheas_pre_common
# district_rheas_prehavest.loc[datasets['district']!='kalomo']

,district,season1,year,yield
0,kalulushi,1,2011,4.06528
1,kalulushi,1,2012,4.01184
2,kalulushi,1,2013,4.15107
3,kalulushi,1,2014,4.00722
4,kalulushi,1,2015,3.87834
...,...,...,...,...
883,chipata,1,2018,3.85177
884,chipata,1,2019,3.69547
885,chipata,1,2020,3.92015
886,chipata,1,2021,4.10539


In [33]:
# rheas and preharvestharvest merged with metrics
# bias correcting rheas using preharvest over the province
rheas_pre_common=rheas_pre_common.groupby(['district'], as_index=False).agg({'yield':'mean'})
preharvest_yearly=preharvest_yearly.groupby(['district'],as_index=False).agg({'yield':'mean'})

# marge preharvest, metrics and rheas rheas yields
preharvest_yearly.rename(columns={'yield':'pre_yield'}, inplace=True)
district_rheas_preharvest_merged=pd.concat([rheas_pre_common,preharvest_yearly,district_rheas_preharvest_metrics], axis=1)

# calculate bias corrected yield
district_rheas_preharvest_merged['corrected_yield']=district_rheas_preharvest_merged['yield']-district_rheas_preharvest_merged['bias']
# district_rheas_preharvest_merged.to_csv('data/rheas_preharvest_district_corrected.csv', index=False)

In [34]:
# Province Level
# rename columns
preharvest['district_b']=preharvest['district']
tamsat_season['district_b']=tamsat_season['district']
postharvest['province_b']=postharvest['province']


#Read shapefiles and rename columns
zambia_district_shp = gpd.read_file('data/shp/zmb_district_cleaned.shp')
zambia_shp = gpd.read_file('../zambia/data/shp/gadm36_ZMB_0.shp')
zambia_province_shp = gpd.read_file('data/shp/zmb_province_cleaned.shp')

# merge the yields and shapefiles by provinces
preharvest_shp= pd.merge(zambia_district_shp, preharvest, on="district_b")
post_shp= pd.merge(zambia_province_shp, postharvest, on="province_b")
rheas_tamsat_shp= pd.merge(zambia_district_shp, tamsat_season, on="district_b")

# aggregate preharvestharvest to provincial level 
preharvest_provincial=preharvest_shp.groupby(['province_b','province', 'year'], as_index=False).agg({'harvested_ha':'sum', 'production_mt': 
                                                             'sum','planted_ha':'sum' })
preharvest_provincial['yield']=preharvest_provincial['production_mt']/preharvest_provincial['harvested_ha']
preharvest_provincial['yield_planted']=preharvest_provincial['production_mt']/preharvest_provincial['planted_ha']
rheas_tamsat_province=rheas_tamsat_shp.groupby(['province_b', 'province', 'year'], as_index=False).agg({'yield':'mean'})
post_province=post_shp.groupby(['province_b', 'year'], as_index=False).agg({'yield':'mean'})
post_province_rheas_post=post_province


In [35]:
provinces=['central', 'copperbelt', 'eastern', 'luapula', 'lusaka',
       'muchinga', 'northern', 'northwestern', 'southern', 'western']
rheas_provincial1=rheas_tamsat_province.loc[(rheas_tamsat_province['year']>=2011)&(rheas_tamsat_province['year']<=2022)]
rheas_provincial1=rheas_provincial1.reset_index()
rheas_preharvestharvest_province_metrics1=[]
rheas_preharvest_prov_check=[]
for i in provinces:
    preharvest_province1=preharvest_provincial.loc[preharvest_provincial['province_b']==i].reset_index(drop=True)
    rheas_tamsat_province1=rheas_provincial1.loc[rheas_provincial1['province_b']==i].reset_index(drop=True)  
    #  pearson's correlation
    corr=rheas_tamsat_province1['yield'].corr(preharvest_province1['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheas_tamsat_province1['yield']-preharvest_province1['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheas_tamsat_province1['yield']-rheas_tamsat_province1['yield'].mean())
                       -(preharvest_province1['yield']-preharvest_province1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheas_tamsat_province1['yield']-preharvest_province1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    rheas_preharvestharvest_province_metrics1.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])
    rheas_preharvest_prov_check.append(preharvest_province1)

check_rheas_preharvest_prov= pd.concat(rheas_preharvest_prov_check, ignore_index=True)
rheas_preharvest_prov_metric=pd.DataFrame(rheas_preharvestharvest_province_metrics1)
rheas_preharvestharvest_province_metric=rheas_preharvest_prov_metric.rename({0:'province',1: 'correlation',2: 
                                                  'rmse', 3:'ubrmse', 4:'bias'}, axis=1)
rheas_preharvestharvest_province_metric
# rheas_provincial1

,province,correlation,rmse,ubrmse,bias
0,central,0.379618,1.189460,0.302376,1.150385
1,copperbelt,-0.061651,1.069403,0.202911,1.049976
2,eastern,0.473370,1.772778,0.162292,1.765333
3,luapula,0.137104,0.986394,0.183999,0.969080
4,lusaka,0.287522,1.203676,0.335756,1.155900
5,muchinga,0.205617,0.536010,0.188335,0.501833
6,northern,0.571940,1.078418,0.225357,1.054608
7,northwestern,-0.390952,1.307819,0.347620,1.260774
8,southern,0.422918,1.811383,0.375846,1.771962
9,western,0.165242,2.651487,0.260853,2.638624


In [36]:
# save province metrics
rheas_preharvestharvest_province_metric.to_csv('data/metrics/preharvest_rheas_province_level_metrics.csv', index=False)

In [37]:
# rheas and preharvestharvest merged with metrics
# bias correcting rheas using preharvest over the province
rheas_provincial1=rheas_provincial1.groupby(['province_b'], as_index=False).agg({'yield':'mean'})
post_province_bias=post_province.groupby(['province_b'],as_index=False).agg({'yield':'mean'})

# marge preharvest, metrics and rheas rheas yields
post_province_renamed=post_province_bias.rename(columns={'yield':'post_yield'})
province_rheas_preharvest_merged=pd.concat([rheas_provincial1,post_province_renamed,rheas_preharvestharvest_province_metric], axis=1)

# calculate bias corrected yield
province_rheas_preharvest_merged['corrected_yield']=province_rheas_preharvest_merged['yield']-province_rheas_preharvest_merged['bias']
province_rheas_preharvest_merged.drop(columns={'province_b'})

,yield,post_yield,province,correlation,rmse,ubrmse,bias,corrected_yield
0,3.972415,2.078341,central,0.379618,1.189460,0.302376,1.150385,2.822031
1,3.968746,2.210160,copperbelt,-0.061651,1.069403,0.202911,1.049976,2.918770
2,3.973352,1.768208,eastern,0.473370,1.772778,0.162292,1.765333,2.208019
3,3.970544,2.554374,luapula,0.137104,0.986394,0.183999,0.969080,3.001463
4,3.974543,2.115439,lusaka,0.287522,1.203676,0.335756,1.155900,2.818643
5,3.986900,2.621986,muchinga,0.205617,0.536010,0.188335,0.501833,3.485066
6,3.980668,2.440774,northern,0.571940,1.078418,0.225357,1.054608,2.926060
7,3.982620,2.216512,northwestern,-0.390952,1.307819,0.347620,1.260774,2.721846
8,3.972499,1.314307,southern,0.422918,1.811383,0.375846,1.771962,2.200537
9,3.976332,0.736167,western,0.165242,2.651487,0.260853,2.638624,1.337707


In [38]:
# # Postharvest and rheas common years 
post_list_years= postharvest['year'].to_list()
years_set=set(post_list_years)
post_harvest_years=list(years_set)
post_harvest_years

# df1=rheas_tamsat_province.loc[rheas_tamsat_province['year']==2016]
rheas_tamsat_province2=pd.DataFrame(columns =[rheas_tamsat_province.columns])

emptylist=[]
for i in post_harvest_years:
    df=rheas_tamsat_province.loc[rheas_tamsat_province['year']==i].reset_index(drop=True)
    emptylist.append(df)
dataset2=pd.concat(emptylist, ignore_index=True)
rheas_tamsat_province
rheas_tamsat_province2
dataset2

,province_b,province,year,yield
0,central,Central,2011,4.002420
1,copperbelt,Copperbelt,2011,4.017213
2,eastern,Eastern,2011,4.002530
3,luapula,Luapula,2011,4.001924
4,lusaka,Lusaka,2011,4.010953
5,muchinga,Muchinga,2011,4.013379
6,northern,Northern,2011,4.004577
7,northwestern,North-Western,2011,4.018557
8,southern,Southern,2011,4.005731
9,western,Western,2011,4.005729


In [39]:
# Postharvest and rheas metrics
# only data from 2011 to 2014
post_province3=post_province_rheas_post
rheas_tamsat_post_province1=dataset2
rheas_post_prov_check=[]
province_metrics2=[]
for i in provinces:
    post_province1=post_province_rheas_post.loc[post_province_rheas_post['province_b']==i].reset_index(drop=True)
    rheas_tamsat_province2=rheas_tamsat_post_province1.loc[rheas_tamsat_post_province1['province_b']==i].reset_index(drop=True)

    #  pearson's correlation
    corr=rheas_tamsat_province2['yield'].corr(post_province1['yield'])

    # #root mean square error
    rmse = np.sqrt(((rheas_tamsat_province2['yield']-post_province1['yield'])**2).mean())
    # #unbiased RMSE
    ubrmse = np.sqrt((((rheas_tamsat_province2['yield']-rheas_tamsat_province2['yield'].mean())
                       -(post_province1['yield']-post_province1['yield'].mean()))**2).mean())
    # #bias
    bias = (rheas_tamsat_province2['yield']-post_province1['yield']).mean()
    # for i in districts:
    # metrics.extend([i, corr])
    province_metrics2.extend([[i, corr,rmse,ubrmse,bias]])
    # metrics.extend([i, corr])
    rheas_preharvest_prov_check.append(preharvest_province1)

check_rheas_preharvest_prov= pd.concat(rheas_preharvest_prov_check, ignore_index=True)

province_statistics2=pd.DataFrame(province_metrics2)
province_statistics2=province_statistics2.rename({0:'province',1: 
                                                  'correlation',2: 'rmse', 3:'ubrmse', 4:'bias'}, axis=1)

province_statistics2
province_statistics2

,province,correlation,rmse,ubrmse,bias
0,central,0.769980,1.962984,0.192172,1.953554
1,copperbelt,0.874979,1.842614,0.110947,1.839270
2,eastern,-0.121522,2.269356,0.091777,2.267499
3,luapula,0.719022,1.480009,0.050387,1.479151
4,lusaka,0.501059,1.923918,0.085281,1.922027
5,muchinga,0.650258,1.440975,0.129569,1.435138
6,northern,0.960482,1.603873,0.157855,1.596086
7,northwestern,0.856748,1.844209,0.234876,1.829191
8,southern,0.290188,2.720767,0.145441,2.716876
9,western,-0.259064,3.307551,0.232796,3.299349


In [40]:
# save postharvest_rheas_province_level_metrics
province_statistics2.to_csv('data/metrics/postharvest_rheas_province_level_metrics.csv', index=False)

In [41]:
# bias correcting rheas using preharvest over the province
rheas_tamsat_post_province_grouped=rheas_tamsat_post_province1.groupby(['province_b'], as_index=False).agg({'yield':'mean'})
post_province=post_province.groupby(['province_b'],as_index=False).agg({'yield':'mean'})
post_province_renamed2=post_province.rename({'yield': 'post_yield'},axis=1)

# marge preharvest, metrics and rheas rheas yields
province_rheas_preharvest=pd.concat([rheas_tamsat_post_province_grouped,
                                     post_province_renamed2,province_statistics2], axis=1)

# calculate bias corrected yield
province_rheas_preharvest['corrected_yield']=(province_rheas_preharvest['yield'])-(province_rheas_preharvest['bias'])
province_rheas_preharvest

,province_b,yield,province_b,post_yield,province,correlation,rmse,ubrmse,bias,corrected_yield
0,central,4.031895,central,2.078341,central,0.769980,1.962984,0.192172,1.953554,2.078341
1,copperbelt,4.049430,copperbelt,2.210160,copperbelt,0.874979,1.842614,0.110947,1.839270,2.210160
2,eastern,4.035708,eastern,1.768208,eastern,-0.121522,2.269356,0.091777,2.267499,1.768208
3,luapula,4.033524,luapula,2.554374,luapula,0.719022,1.480009,0.050387,1.479151,2.554374
4,lusaka,4.037466,lusaka,2.115439,lusaka,0.501059,1.923918,0.085281,1.922027,2.115439
5,muchinga,4.057124,muchinga,2.621986,muchinga,0.650258,1.440975,0.129569,1.435138,2.621986
6,northern,4.036860,northern,2.440774,northern,0.960482,1.603873,0.157855,1.596086,2.440774
7,northwestern,4.045702,northwestern,2.216512,northwestern,0.856748,1.844209,0.234876,1.829191,2.216512
8,southern,4.031184,southern,1.314307,southern,0.290188,2.720767,0.145441,2.716876,1.314307
9,western,4.035516,western,0.736167,western,-0.259064,3.307551,0.232796,3.299349,0.736167


In [42]:
# bias correction

# rheas_tamsat_province['yield_corrected']
rheas_tamsat_preharvest_province=rheas_tamsat_province.groupby(['province_b'], as_index=False).agg({'yield':'mean'})
post_province=post_province.groupby(['province_b'], as_index=False).agg({'yield':'mean'})
post_province.rename({'province_b':'province_p','yield':'postharvest'},axis=1, inplace=True)
rheas_tamsat_preharvest_province['yield_corrected']=rheas_tamsat_preharvest_province['yield']-province_statistics2['bias']
rheas_postharvest_provincial=pd.concat([rheas_tamsat_preharvest_province,post_province,province_statistics2], axis=1)

rheas_postharvest_provincial1=rheas_postharvest_provincial[['province','yield','postharvest', 'yield_corrected','correlation', 
                              'rmse', 'ubrmse', 'bias']]
rheas_postharvest_provincial1.rename({'yield':'rheas'},axis=1, inplace=True)
rheas_postharvest_provincial1

C:\Users\kasit\AppData\Local\Temp\ipykernel_16656\3299489503.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rheas_postharvest_provincial1.rename({'yield':'rheas'},axis=1, inplace=True)


,province,rheas,postharvest,yield_corrected,correlation,rmse,ubrmse,bias
0,central,4.009854,2.078341,2.056300,0.769980,1.962984,0.192172,1.953554
1,copperbelt,3.997637,2.210160,2.158366,0.874979,1.842614,0.110947,1.839270
2,eastern,4.008018,1.768208,1.740519,-0.121522,2.269356,0.091777,2.267499
3,luapula,4.012911,2.554374,2.533760,0.719022,1.480009,0.050387,1.479151
4,lusaka,4.008398,2.115439,2.086371,0.501059,1.923918,0.085281,1.922027
5,muchinga,4.018974,2.621986,2.583837,0.650258,1.440975,0.129569,1.435138
6,northern,4.019895,2.440774,2.423809,0.960482,1.603873,0.157855,1.596086
7,northwestern,4.016192,2.216512,2.187001,0.856748,1.844209,0.234876,1.829191
8,southern,4.009252,1.314307,1.292376,0.290188,2.720767,0.145441,2.716876
9,western,4.017791,0.736167,0.718442,-0.259064,3.307551,0.232796,3.299349
